# 2.3 Example - XANES LCF using Experiment Class

    In this example a first-middle-last scan LCF fit will be compelted and plotted against the known process parameters.

    This example assumes that you have a pickled experiment by previoudly running:
        '2.0 Example - Experiment Class -  XAS Data Processing'
        '2.1 Example - Experiment Class -  Normalized XANES-EXAFS-FT'
        '2.2 Example - Experiment Class -  XAS-Process Stream Correlation'


### Steps:
    
    1. Import Modules
    2. Define Paths
    3. Open Saved Experiment using 'pickle'
    4. Perform LCF
    5. Save Experiment with 'pickle'

### Things to have ready when attemtping this example:

    1. Directory and filename for the 'pickled' experiment 
    2. Scan name of one spectra in the experiment object

## Step 1. Import Modules

In [1]:
# File Handling
import os
import pickle

# Make Plotting interactive
from ipywidgets import interact, fixed

# Wrapper for xraylarch (larch)
os.chdir('..')
pwd = os.getcwd()
os.chdir('./catxas')
import general as fcts
import xas as xfcts
import plot as pfcts
import process
import experiment as exp

# Plot Handling
%matplotlib inline

## Step 2. Define Paths

    Note: The sample shown here has mass spec and LabView data.  As more process streams are generated, additional formats can be added. This is just an example.

#### Modify:
    1. pickle_dir - directory containing the pickled experiment
    2. pickpe_name - filename of the pickled experiment (must include extension '.pickle')
    3. scan_name - a name of one fo the spectra in the experiment, used for interrogation throughout

In [2]:
# Directory Containing Experiment:
pickle_dir = os.path.join(pwd, 'sample results')

# Name of Experiment (must include extension '.pickle')
pickle_name = 'SnO2_TPR.pickle'

# Scan Naame for looking at parts of the Experiment Object
scan_name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001'

## Step 3. Open Saved Experiment using 'pickle'

    Pickle - A method to save the experimental class and reopen it elsewhere.

In [3]:
pickle_path = os.path.join(pickle_dir,pickle_name)
pickle_in = open(pickle_path,"rb")
my_exp = pickle.load(pickle_in)
pickle_in.close()

## Step 4. Perform LCF

    Notes:
        a. Basis sets must be larch groups that have been calibrated and normalized
        

### Major steps for performing LCF:
    
    a. Generate list of basis spectra groups
    b. Load basis set into a "Fit"
    c. Perform LCF
    d. Generate a Fit Report
    e. Visualize Fit and Errors with one Process Parameter
    f. Visulaize Single Fit
    


### 4a. Generate list of basis spectra groups

    In this section we ae using three basis spectra that are taken from the main data set, not standards.
        1. First Scan (*_0001)
        2. ~Middle Scan (*_0003)
        3. Last Scan (*_0050)
        
    Notes:
        1. Each basis gropu must have its own name.
        2. Basis are compiled into a list prior to uploading into the experiment

In [4]:
# Basis set made from three spectra in the sample dataset
basis1 = my_exp.spectra['20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001']['Absorption Spectra']['mu Sample'] 
basis1.name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001'

basis2 = my_exp.spectra['20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0031']['Absorption Spectra']['mu Sample']
basis2.name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0031'

basis3 = my_exp.spectra['20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0050']['Absorption Spectra']['mu Sample']
basis3.name = '20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0050'

basis = [basis1, basis2, basis3]

### 4b. Load basis set into a "Fit"

    Each LCF fit can be saved under a different "Fit name". Choosing the same name will overwrite a former Fit

#### MODIFY:
    1. fit_name - unique name for the fit

In [5]:
# Load list of basis spectra into the experiment
fit_name = 'Fit 1'


my_exp.load_lcf_basis(basis, 'Fit 1')

#### Inspection tool to show how LCF is added to the experiment
    1. Dictionary of fits - experiment.analysis['LCF]
    2. Visualization of the uploaded basis sets

In [6]:
# 1. Inspect Analysis side of experiment

print('keys in the experiment.analysis object:')
for key in my_exp.analysis.keys():
    print(f'\t{key}')
    
print('\nkeys in the "LCF" dictionary')
for key in my_exp.analysis['LCF'].keys():
    print(f'\t{key}')

print('\nkeys of "Fit 1"')
for key in my_exp.analysis['LCF']['Fit 1'].keys():
    print(f'\t{key}')  

keys in the experiment.analysis object:
	LCF
	PCA

keys in the "LCF" dictionary
	Fit 1

keys of "Fit 1"
	basis spectra


In [7]:
# 2. Visualize basis sets for fitting

# MODIFY: fit_name, emin, emax

fit_name = 'Fit 1'
emin = 29150
emax = 29300


basis_groups = my_exp.analysis['LCF'][fit_name]['basis spectra']

pfcts.plot_XANES(basis_groups, emin, emax, spectra = 'flat', 
                 deriv = False, e0 = None, e0_line = False, 
                 overlay = True, use_legend = False)

### 4c. Perform LCF

    This fitting step allows different paramters to updated and the fit to be performed.  This step can be run multiple times as needed.

#### MODIFY:
    1. fit_name - which fit is being completed 
    2. emin - starting energy range for the LCF
    3. emax - ending enregy range for the LCF
    4. minvals - minimum values that fits can be at (lower limit of component fraction)
    5. maxvals - maximum values that fits can be at (upper limit of component fraction)
    6. array_name - what spectra to fit, 'mu', 'norm', 'flat' are common options
    7. sum_to_one - True = all components fractions must sum to 1, False = component fractions can some to anything
    

In [8]:
# Fit the sample spectra
fit_name = 'Fit 1'
emin = 29150
emax = 29250
minvals = 0
maxvals = 1
array_name = 'flat'
sum_to_one = True

my_exp.fit_LCF(fit_name, emin, emax, 
               weights = None, minvals = minvals, maxvals = maxvals, 
               arrayname = array_name, sum_to_one = sum_to_one)

#### Inspection tool to show how LCF is updated
    1. Dictionary of fit - experiment.analysis['LCF'][fit_name]
    2. LCF result for a single spectra

In [9]:
# 1. inspection of the updated fit 

print('keys of "Fit 1"')
for key in my_exp.analysis['LCF']['Fit 1'].keys():
    print(f'\t{key}') 

keys of "Fit 1"
	basis spectra
	Results


In [10]:
# 2. LCF results for the first scan 
my_exp.analysis['LCF']['Fit 1']['Results'][scan_name]

<Group 0x1b46d40cb50>

### 4d. Generate a Fit Report

    This step compiles all the LCF results for each spectr into a easy to read dataframe into "Fit Summary"
    
#### Modify:
    
    1. fit_name - name of fit to generate report on

In [11]:
# Generate fit report for all spectra
fit_name = 'Fit 1'

my_exp.lcf_report('Fit 1')

#### Inspection tool to show how LCF is updated
    1. Dictionary of fit - experiment.analysis['LCF'][fit_name]
    2. Fit Summary results

In [12]:
# 1. inspection of the updated fit contents

print('keys of "Fit 1"')
for key in my_exp.analysis['LCF']['Fit 1'].keys():
    print(f'\t{key}')

keys of "Fit 1"
	basis spectra
	Results
	Fit Summary


In [13]:
# 2. Contents of the Fit Summary
my_exp.analysis['LCF']['Fit 1']['Fit Summary']

Name          Chi2       RedChi2  \
0   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001  3.547138e-29  9.142108e-32   
1   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0002  4.068672e-03  1.051336e-05   
2   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0003  2.713975e-03  7.012854e-06   
3   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0004  3.277610e-03  8.447450e-06   
4   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0005  3.754680e-03  9.677011e-06   
5   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0006  3.846959e-03  9.914842e-06   
6   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0007  3.487603e-03  9.011894e-06   
7   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0008  2.756053e-03  7.121583e-06   
8   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0009  3.873673e-03  1.000949e-05   
9   20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0010  4.469716e-03  1.154965e-05   
10  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0011  3.444798e-03  8.901286e-06   
11  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0012  4.523410e-03  1.168840e-05   
12  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0013  5.221962e-03  1.345867e-05   
13  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0014  4.312844e-03  1.114430e-05   
14  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0015  6.012001e-03  1.553489e-05   
15  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0016  4.753204e-03  1.228218e-05   
16  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0017  6.393395e-03  1.652040e-05   
17  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0018  4.598053e-03  1.188127e-05   
18  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0019  6.130243e-03  1.584042e-05   
19  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0020  4.948301e-03  1.278631e-05   
20  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0021  6.737104e-03  1.740854e-05   
21  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0022  6.270365e-03  1.620249e-05   
22  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0023  9.087367e-03  2.348157e-05   
23  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0024  5.244980e-03  1.355292e-05   
24  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0025  7.013904e-03  1.812378e-05   
25  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0026  6.214829e-03  1.610059e-05   
26  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0027  7.914280e-03  2.045033e-05   
27  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0028  4.187301e-03  1.081990e-05   
28  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0029  2.653022e-03  6.855353e-06   
29  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0030  2.301066e-03  5.945906e-06   
30  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0031  1.456175e-28  3.762727e-31   
31  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0032  1.167718e-02  3.017358e-05   
32  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0033  2.673579e-02  6.908472e-05   
33  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0034  2.171306e-02  5.596151e-05   
34  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0035  3.066795e-01  7.924536e-04   
35  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0036  1.094035e-02  2.819679e-05   
36  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0037  1.076651e-01  2.782044e-04   
37  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0038  5.455855e-02  1.409782e-04   
38  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0039  3.013537e-02  7.786917e-05   
39  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0040  8.032668e-03  2.075625e-05   
40  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0041  5.958918e-03  1.539772e-05   
41  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0042  3.569185e-03  9.222699e-06   
42  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0043  4.075412e-03  1.053078e-05   
43  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0044  2.883912e-03  7.451970e-06   
44  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0045  2.617167e-03  6.762707e-06   
45  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0046  2.862044e-03  7.376403e-06   
46  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0047  3.998169e-03  1.033119e-05   
47  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0048  3.673689e-03  9.492737e-06   
48  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0049  2.183514e-03  5.642154e-06   
49  20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0050  5.165249e-30  1.334690e-32   

    Variables          Amp1    Amp1-stdev          Amp2    Amp2-stdev  \
0           2  1.000000e+00  2.689348e-1

### 4e. Visualize Fit and Errors with one Process Parameter

#### Modify:
    1. fit_name
    2. process_param - name of parameter from a process stream. Common option is temperature (from LV:'TR1 T [°C]')

In [14]:
# Visualize Fit with process parameter

fit_name = 'Fit 1'
process_param = 'TR1 T [°C]'

my_exp.plot_LCF_results(fit_name, process_parameter = process_param)

### 4f. Visulaize Single Fit

#### Modify:
    1. fit_name
    2. emin/emax - Energy range to plot over 
    3. ymin/ymax - Adjsutd scaling as needed

In [15]:
# Visualize individual fits

fit_name = 'Fit 1'
emin = 29150
emax = 29250
ymin = -0.05
ymax = 1.6


interact(my_exp.plot_LCF, spectra = my_exp.spectra.keys(),
        fit_name = fixed(fit_name),
        emin = fixed(emin), emax = fixed(emax), 
        ymin = fixed(ymin), ymax = fixed(ymax))



interactive(children=(Dropdown(description='spectra', options=('20210614_SnO2_H2_TPR_Sn_EXAFS_92.tra_0001', '2…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

## Step 5. Save Experiment with 'pickle' 
    Pickle - A method to save the experimental class and reopen it elsewhere.

In [16]:
pickle_out = open(pickle_path,"wb")
pickle.dump(my_exp, pickle_out)
pickle_out.close()